#### 給定隊伍等級與裁判隊伍自動分組與更新ref_conflict表

In [6]:
import pandas as pd
import random

# === 讀取 Excel 資料 ===
excel_data = pd.read_excel("referees.xlsx", sheet_name=None)
level_df = excel_data["level"]
ref_team_df = excel_data["ref_team"]

# === 整理資料 ===
# Get teams and their levels
level_df = level_df.rename(columns={"team_name": "team", "level": "level"})
team_levels = level_df.set_index("team")["level"].to_dict()

# Randomly assign teams to groups
teams_by_level = {level: [] for level in range(1, 5)}  # Levels 1 to 4
for team, level in team_levels.items():
    teams_by_level[level].append(team)

# Check team availability for groups A to H (levels 1 to 4)
required_teams_level_1_to_4 = 8  # 8 groups (A to H)
for level in range(1, 5):
    if len(teams_by_level[level]) < required_teams_level_1_to_4:
        raise ValueError(f"Not enough teams at level {level}: need {required_teams_level_1_to_4}, but only {len(teams_by_level[level])} available.")

grouped_teams = {}
# Groups A to H (one from each level 1 to 4)
for group in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
    selected_teams = [random.choice(teams_by_level[level]) for level in range(1, 5)]
    for team in selected_teams:
        teams_by_level[team_levels[team]].remove(team)
    grouped_teams[group] = selected_teams

# Group I (3 teams, prefer one from levels 1, 2, 3 if possible)
remaining_teams = []
for level in range(1, 5):
    remaining_teams.extend(teams_by_level[level])

if len(remaining_teams) < 3:
    raise ValueError(f"Not enough remaining teams for group I: need 3, but only {len(remaining_teams)} available.")

# Try to select one team from each of levels 1, 2, 3
selected_teams = []
available_levels = [level for level in range(1, 4) if teams_by_level[level]]
if len(available_levels) >= 3:
    for level in available_levels[:3]:
        if teams_by_level[level]:
            team = random.choice(teams_by_level[level])
            selected_teams.append(team)
            teams_by_level[level].remove(team)
            remaining_teams.remove(team)
else:
    # Fallback: randomly select 3 teams from remaining teams
    selected_teams = random.sample(remaining_teams, 3)
    for team in selected_teams:
        if team in remaining_teams:
            remaining_teams.remove(team)
            level = team_levels[team]
            if team in teams_by_level[level]:
                teams_by_level[level].remove(team)

grouped_teams['I'] = selected_teams

# Group J (all remaining teams, regardless of level)
remaining_teams = []
for level in range(1, 5):
    remaining_teams.extend(teams_by_level[level])

if remaining_teams:
    grouped_teams['J'] = remaining_teams
    if len(remaining_teams) < 2:
        print(f"⚠️ 警告：Group J 只有 {len(remaining_teams)} 支隊伍，無法形成比賽。")
else:
    print("⚠️ 警告：沒有剩餘隊伍可分配到 Group J。")

# Create DataFrame for groupings
grouping_data = []
for group, teams in grouped_teams.items():
    for team in teams:
        grouping_data.append({"Group": group, "Team": team, "Level": team_levels[team]})

grouping_df = pd.DataFrame(grouping_data)

# === Generate Referee Conflict Table ===
# Get unique referees from ref_team
referees = ref_team_df["name"].unique()

# Initialize conflict dictionary
ref_conflicts = {}
for ref in referees:
    ref_conflicts[ref] = {}

# Fill conflict table: 0 if referee has conflict with any team in group, 1 otherwise
for ref in referees:
    conflicted_teams = set(ref_team_df[ref_team_df["name"] == ref]["dept"])
    for group, teams in grouped_teams.items():
        # Check if referee has conflict with any team in the group
        has_conflict = any(team in conflicted_teams for team in teams)
        ref_conflicts[ref][group] = 0 if has_conflict else 1

# Convert to DataFrame
ref_conflict_df = pd.DataFrame(ref_conflicts).T
ref_conflict_df.index.name = "Referee"

# === 輸出到 Excel ===
with pd.ExcelWriter("group_generate.xlsx", engine="openpyxl") as writer:
    grouping_df.to_excel(writer, sheet_name="Groupings", index=False)
    ref_conflict_df.to_excel(writer, sheet_name="Referee Conflicts")

print("✅ 分組和裁判衝突表已生成！")
print("📤 已輸出：group_generate.xlsx（包含 'Groupings' 和 'Referee Conflicts' 兩頁）")

✅ 分組和裁判衝突表已生成！
📤 已輸出：group_generate.xlsx（包含 'Groupings' 和 'Referee Conflicts' 兩頁）


### 排賽程

In [7]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
from itertools import combinations

# === 讀取 Excel 資料 ===
# Load groupings and referee conflicts
group_data = pd.read_excel("group_generate.xlsx", sheet_name=None)
grouping_df = group_data["Groupings"]
ref_conflict_df = group_data["Referee Conflicts"]

# Load referee and team availability
avail_data = pd.read_excel("臺大盃女排資料表.xlsx", sheet_name=None)
ref_df = avail_data["ref_available"]
teams_df = avail_data["team_available"]

# === 整理資料 ===
# Process referee availability (keep original values: 0, 0.5, 1)
ref_df = ref_df.set_index("name").T
referees = list(ref_df.columns)
days = list(ref_df.index.astype(int))
fields = list(range(4))

# Process referee conflicts
ref_conflict_df = ref_conflict_df.set_index("Referee")
referee_conflicts = ref_conflict_df.to_dict("index")
for ref in referee_conflicts:
    referee_conflicts[ref] = {group: int(value) for group, value in referee_conflicts[ref].items()}

# Process team availability
teams_df = teams_df.rename(columns={"Unnamed: 0": "team"})
team_week_avail = teams_df.set_index("team").to_dict("index")

# Process groupings
grouped_teams = {}
for group, group_df in grouping_df.groupby("Group"):
    grouped_teams[group] = list(group_df["Team"])

# Get list of all teams
teams = [team for group in grouped_teams.values() for team in group]

# Map days to weekdays and define weeks
weekdays = ["mon", "tue", "wed", "thur", "fri"]
day_to_weekday = {d: weekdays[d % 5] for d in days if d % 5 < 5}
valid_days = [d for d in days if d % 5 < 5]

# === 建立比賽清單（照分組，單循環）===
matches = []
for group, team_list in grouped_teams.items():
    if len(team_list) < 2:
        print(f"⚠️ 警告：Group {group} 只有 {len(team_list)} 支隊伍，無法形成比賽，將不會出現在賽程表中。")
        continue
    for t1, t2 in combinations(team_list, 2):
        matches.append((group, t1, t2))

# === 建立 Gurobi 模型 ===
m = Model("volleyball_schedule")
m.setParam('OutputFlag', 0)

# Decision variables
x = {}
for group, t1, t2 in matches:
    for d in valid_days:
        wd = day_to_weekday[d]
        if team_week_avail[t1][wd] and team_week_avail[t2][wd]:
            for f in fields:
                for r in referees:
                    if ref_df.loc[d, r] >= 0.5:
                        if referee_conflicts[r][group] == 1:
                            x[(group, t1, t2, d, f, r)] = m.addVar(vtype=GRB.BINARY)

makespan = m.addVar(vtype=GRB.INTEGER)

# Variables to track the number of games each referee officiates
referee_games = {}
for r in referees:
    referee_games[r] = m.addVar(vtype=GRB.INTEGER, name=f"referee_games_{r}")

# Constraint: calculate the number of games each referee officiates
for r in referees:
    m.addConstr(
        referee_games[r] == quicksum(x.get((g, t1, t2, d, f, rr), 0)
                                    for (g, t1, t2, d, f, rr) in x if rr == r),
        name=f"referee_games_count_{r}"
    )

# Variables to track deviation from the average number of games
total_matches = len(matches)
avg_games = total_matches / len(referees) if referees else 0
deviation_plus = {r: m.addVar(vtype=GRB.CONTINUOUS, name=f"dev_plus_{r}") for r in referees}
deviation_minus = {r: m.addVar(vtype=GRB.CONTINUOUS, name=f"dev_minus_{r}") for r in referees}

# Constraints for deviation
for r in referees:
    m.addConstr(referee_games[r] == avg_games + deviation_plus[r] - deviation_minus[r])

# === 限制條件 ===
# 一場比賽只能排一次
for group, t1, t2 in matches:
    m.addConstr(quicksum(x.get((group, t1, t2, d, f, r), 0)
                         for d in valid_days for f in fields for r in referees
                         if (group, t1, t2, d, f, r) in x) == 1)

# 同一場地每天至多一場
for d in valid_days:
    for f in fields:
        m.addConstr(quicksum(x.get((g, t1, t2, dd, ff, r), 0)
                             for (g, t1, t2, dd, ff, r) in x
                             if dd == d and ff == f) <= 1)

# 同一裁判每天至多執法一場
for d in valid_days:
    for r in referees:
        m.addConstr(quicksum(x.get((g, t1, t2, dd, f, rr), 0)
                             for (g, t1, t2, dd, f, rr) in x
                             if dd == d and rr == r) <= 1)

# 每隊每天至多一場比賽
for t in teams:
    for d in valid_days:
        m.addConstr(quicksum(x.get((g, t1, t2, dd, f, r), 0)
                             for (g, t1, t2, dd, f, r) in x
                             if dd == d and (t1 == t or t2 == t)) <= 1)

# Teams may not play more than twice a week
for t in teams:
    for week_start in range(min(valid_days), max(valid_days) - 4, 5):
        week_days = [d for d in valid_days if week_start <= d < week_start + 5]
        m.addConstr(quicksum(x.get((g, t1, t2, dd, f, r), 0)
                             for (g, t1, t2, dd, f, r) in x
                             if dd in week_days and (t1 == t or t2 == t)) <= 2)

# Teams should not play on consecutive days
for t in teams:
    for d in valid_days[:-1]:  # Exclude last day to avoid index out of range
        next_day = d + 1
        if next_day in valid_days:
            m.addConstr(quicksum(x.get((g, t1, t2, dd, f, r), 0)
                                 for (g, t1, t2, dd, f, r) in x
                                 if dd == d and (t1 == t or t2 == t)) +
                        quicksum(x.get((g, t1, t2, dd, f, r), 0)
                                 for (g, t1, t2, dd, f, r) in x
                                 if dd == next_day and (t1 == t or t2 == t)) <= 1)

# makespan 控制比賽結束日期
for key in x:
    group, t1, t2, d, f, r = key
    m.addConstr(x[key] * d <= makespan)

# === 目標函數 ===
penalty = quicksum(0.1 * x[key] for key in x if ref_df.loc[key[3], key[5]] == 0.5)
balance_penalty = quicksum(deviation_plus[r] + deviation_minus[r] for r in referees)
m.setObjective(makespan + penalty + 0.01 * balance_penalty, GRB.MINIMIZE)

# === 執行求解 ===
m.optimize()

# === 輸出每日賽程表 ===
if m.status == GRB.OPTIMAL:
    result = [(group, t1, t2, d, f, r)
              for (group, t1, t2, d, f, r), var in x.items() if var.X > 0.5]
    df = pd.DataFrame(result, columns=["Group", "Team1", "Team2", "Day", "Field", "Referee"])

    # 建立每日賽程表
    df["Match"] = df["Team1"] + " vs " + df["Team2"]
    schedule_df = df[["Day", "Field", "Match", "Referee"]].sort_values(by=["Day", "Field"])

    # Calculate referee workload
    referee_counts = {r: 0 for r in referees}
    for _, row in df.iterrows():
        referee_counts[row["Referee"]] += 1
    referee_counts_df = pd.DataFrame.from_dict(referee_counts, orient='index', columns=["Games Officiated"])
    referee_counts_df.index.name = "Referee"

    # 輸出到 Excel
    with pd.ExcelWriter("volleyball_schedule.xlsx", engine="openpyxl") as writer:
        schedule_df.to_excel(writer, sheet_name="Schedule", index=False)
        grouping_df.to_excel(writer, sheet_name="Groupings", index=False)
        referee_counts_df.to_excel(writer, sheet_name="Referee Counts")

    print("✅ 排程成功！最晚比賽日：", int(makespan.X))
    print("📊 裁判執法場次統計：")
    for ref, count in referee_counts.items():
        print(f"{ref}: {count} 場")
    print("📤 已輸出：volleyball_schedule.xlsx（包含 'Schedule'、'Groupings' 和 'Referee Counts' 三頁）")
else:
    print("❌ 無可行排程")

Set parameter Username
Set parameter LicenseID to value 2661855
Academic license - for non-commercial use only - expires 2026-05-07
✅ 排程成功！最晚比賽日： 14
📊 裁判執法場次統計：
小馬: 4 場
恩臨: 3 場
羿君: 3 場
茵茵: 3 場
芳芳: 4 場
阿冠: 4 場
小魚: 0 場
yoyo: 4 場
家葳: 5 場
絲瓜: 5 場
大餅: 4 場
手槍: 4 場
阿程: 2 場
阿侑: 5 場
阿宛: 4 場
📤 已輸出：volleyball_schedule.xlsx（包含 'Schedule'、'Groupings' 和 'Referee Counts' 三頁）
